## SWE for DS Chap 1 Waht is good code

- what is good code
>- simplicity - don't repeat yourself; all knowledge should have one single representation in code; drop unnecessary complexity or unneeded lines
>- modularity - code skeleton, functions 
>- readability - adhering to standards and conventions for your programming language, choosing good names, removing unused code, and writing documentation for your code, 
>- performance - runtime and memory, 
>- robustness - reproducible; run without failure + responds to changes gracefully; properly handling errors, logging what has happened, and writing good tests; testing (user testing & automated testing)

- tech debt
>- for deferred work resulting from when code is written quickly instead of correctly. missing documentation, poorly structured code, poorly named variables, or any other cut corners. although, it's often necessary because of business deadlines and budgets

- create skeleton code
```
def load_data (csv_file):
    pass

def clean_data(input_data, max_length):
    pass
```

- common testing of code
>- Unit tests test a single function
>- end-to-end tests test a whole project
>- integration tests test a chunk of code that contains many functions but is still smaller than a whole project

## SWE for DS Chap 2 Analyzing code performance

- Performant code: works; returns output in a timely manner; not exceeds computing resources available
- Figure out the optimization requirements before doing it
- Measure performance of the code: time and space complexity; times it takes to run the code and memeory footprint of the code; often there’s a trade-off where faster code takes upmore memory
- Methods to improve performance:
>- choice of algorithm: e.g., avoid nested loops where possible
>- choice of data structure: e.g., it’s much quicker to look up avalue in a dictionary than to search for it in a list (more in Chap 3)
>- Using built-in functions
>- Compiling Python: by compiling Python to alower-level language using tools like Cython, Numba, and PyPy
>>- Numba contains a subset of Python
>>- Cython is a superset of Python with additional options in C
>>- PyPy is a reimplementation of Python using just-in-time compilation
>- Asynchronous code: speed up your code by having it accomplish a task whileit waits for the outcome of another
>- Parallel and distributed computing
>>- parallel: on multiple CPUs at once using the multiprocessing module; multiple GPU
>>- distributed: on multiple different machines at the same time 

#### Timing the code